<a href="https://colab.research.google.com/github/SaraACHKAOU/E-banking/blob/main/CIFAR_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import numpy as np
import time
import pandas as pd
!pip install keras
from keras.utils import to_categorical  # Remplace np_utils par to_categorical
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Input, Dense, Flatten, Dropout, Concatenate, Activation
from keras.layers import BatchNormalization
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split


In [60]:
train=pd.read_csv('train.csv',sep=",")
test=pd.read_csv('test.csv',sep=",")

In [61]:
train.head()

,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,pixel_9,...,pixel_3063,pixel_3064,pixel_3065,pixel_3066,pixel_3067,pixel_3068,pixel_3069,pixel_3070,pixel_3071,label
0,59,43,50,68,98,119,139,145,149,149,...,58,65,59,46,57,104,140,84,72,6
1,154,126,105,102,125,155,172,180,142,111,...,42,67,101,122,133,136,139,142,144,9
2,255,253,253,253,253,253,253,253,253,253,...,83,80,69,66,72,79,83,83,84,9
3,28,37,38,42,44,40,40,24,32,43,...,39,59,42,44,48,38,28,37,46,4
4,170,168,177,183,181,177,181,184,189,189,...,88,85,82,83,79,78,82,78,80,1


In [62]:
test.head()

,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,pixel_9,...,pixel_3062,pixel_3063,pixel_3064,pixel_3065,pixel_3066,pixel_3067,pixel_3068,pixel_3069,pixel_3070,pixel_3071
0,158,159,165,166,160,156,162,159,158,159,...,130,123,145,167,182,175,145,124,129,110
1,235,231,232,232,232,232,232,232,232,232,...,108,117,123,133,141,153,163,178,191,199
2,158,158,139,132,166,182,187,193,199,205,...,45,46,44,44,43,52,37,8,3,7
3,155,167,176,190,177,166,168,166,170,179,...,52,55,70,103,105,72,53,50,52,50
4,65,70,48,30,23,40,44,45,45,40,...,102,127,156,139,131,130,147,136,146,117


In [63]:
np.shape(test)
np.shape(train)

(50000, 3073)

In [64]:
x_train = train.iloc[:,0:3072]
x_test = test.iloc[:,0:3072]


In [65]:
y_train = train[['label']]
#y_test =  test[['label']]

In [66]:

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.3, shuffle=True)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, shuffle = True)

In [67]:
np.shape(x_train), np.shape(x_test), np.shape(x_val)

((35000, 3072), (7500, 3072), (7500, 3072))

In [68]:
# Normalisation
x_train /= 255
x_test /= 255
x_val /= 255

In [69]:
from tensorflow.python.keras.utils import np_utils
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)
Y_val= np_utils.to_categorical(y_val, 10)

In [70]:
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()
x_val = x_val.to_numpy()

In [71]:
np.shape(x_val)

(7500, 3072)

In [72]:
X_train = x_train.reshape(x_train.shape[0], 32, 32, 3)
X_test = x_test.reshape(x_test.shape[0], 32, 32, 3)
X_val = x_val.reshape(x_val.shape[0], 32, 32, 3)

In [73]:
X_val[0]

array([[[0.83137255, 0.52156863, 0.27843137],
        [0.30196078, 0.30980392, 0.33333333],
        [0.34901961, 0.38039216, 0.42745098],
        ...,
        [0.54901961, 0.75294118, 0.87058824],
        [0.87843137, 0.82745098, 0.82352941],
        [0.80784314, 0.76862745, 0.78431373]],

       [[0.79215686, 0.30196078, 0.16470588],
        [0.24705882, 0.24705882, 0.24705882],
        [0.28235294, 0.30588235, 0.28627451],
        ...,
        [0.40784314, 0.52156863, 0.67843137],
        [0.79607843, 0.85098039, 0.87058824],
        [0.8745098 , 0.89019608, 0.90588235]],

       [[0.2745098 , 0.31764706, 0.56078431],
        [0.75686275, 0.78823529, 0.7372549 ],
        [0.81960784, 0.74901961, 0.60784314],
        ...,
        [0.76470588, 0.88627451, 0.78431373],
        [0.72941176, 0.7372549 , 0.62745098],
        [0.57647059, 0.70980392, 0.84705882]],

       ...,

       [[0.43921569, 0.4745098 , 0.38431373],
        [0.49803922, 0.64313725, 0.24705882],
        [0.18431373, 0

In [74]:
batch_size = 512
num_epochs = 400

model = Sequential()

model.add(Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=(32,32, 3), padding='same', strides=1))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(128, kernel_size=(3,3), activation='relu', padding='same', strides=1))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(120, activation='relu'))

model.add(Dense(10, activation='softmax'))



/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [89]:
checkpointer = ModelCheckpoint(
    filepath='CNN_model_cifar.keras',
    monitor='val_loss', verbose=0,
     save_best_only=True,
    save_weights_only=False,
    mode='min',
   save_freq='epoch')

In [90]:
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                   epochs=num_epochs,
                   batch_size=batch_size, validation_data=(X_val, Y_val), callbacks=[checkpointer, early])

Epoch 1/400
69/69 ━━━━━━━━━━━━━━━━━━━━ 178s 3s/step - accuracy: 0.4264 - loss: 1.6736 - val_accuracy: 0.5036 - val_loss: 1.4243
Epoch 2/400
69/69 ━━━━━━━━━━━━━━━━━━━━ 211s 3s/step - accuracy: 0.5021 - loss: 1.4205 - val_accuracy: 0.5156 - val_loss: 1.3774
Epoch 3/400
69/69 ━━━━━━━━━━━━━━━━━━━━ 198s 3s/step - accuracy: 0.5313 - loss: 1.3354 - val_accuracy: 0.5404 - val_loss: 1.3238
Epoch 4/400
69/69 ━━━━━━━━━━━━━━━━━━━━ 175s 3s/step - accuracy: 0.5527 - loss: 1.2860 - val_accuracy: 0.5769 - val_loss: 1.2268
Epoch 5/400
69/69 ━━━━━━━━━━━━━━━━━━━━ 202s 3s/step - accuracy: 0.5795 - loss: 1.2127 - val_accuracy: 0.5632 - val_loss: 1.2278
Epoch 6/400
69/69 ━━━━━━━━━━━━━━━━━━━━ 202s 3s/step - accuracy: 0.5869 - loss: 1.1899 - val_accuracy: 0.5999 - val_loss: 1.1492
Epoch 7/400
69/69 ━━━━━━━━━━━━━━━━━━━━ 186s 3s/step - accuracy: 0.6070 - loss: 1.1322 - val_accuracy: 0.6119 - val_loss: 1.1262
Epoch 8/400
69/69 ━━━━━━━━━━━━━━━━━━━━ 192s 3s/step - accuracy: 0.6152 - loss: 1.1083 - val_accuracy: 0.

KeyboardInterrupt: 